<a href="https://colab.research.google.com/github/dmsophia/prova_mod10/blob/main/C%C3%B3pia_de_ITL_SI10_2024_Prova_P1_Sophia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
  <img src="https://www.inteli.edu.br/wp-content/uploads/2022/04/28103439/Logo-Container.png" />
</center>

# `[SI10-2024-P1]` - Prova Prática

Sistemas de Informação - Módulo 10 - Turma 4



**Nome:** Sophia Mello Dias

**ANTES DE PROSSEGUIR FAÇA UMA CÓPIA DESTE NOTEBOOK**

----

## Contexto

Uma plataforma online, que atua na venda de produtos, deseja otimizar sua interface para melhorar a experiência do usuário e aumentar as taxas de conversão. Recentemente, a empresa implementou um redesign de sua página web, e os registros dessa mudança estão detalhados nos dados coletados desde então. As variáveis registradas diariamente incluem o número de visitantes únicos, sessões, taxa de rejeição, páginas por sessão, principais origens do tráfego, tipos de dispositivos usados, eventos ocorridos, taxa de conversão e o tempo médio de sessão.

Com base nesses dados, a empresa considera realizar uma mudança similar para otimizar ainda mais sua plataforma. Antes de proceder, a equipe deseja empregar simulações de Monte Carlo para avaliar os possíveis impactos de um novo redesign na taxa de conversão, tempo médio de sessão e taxa de rejeição, utilizando análises estatísticas para prever tendências futuras e ajustar estratégias de maneira informada. Essa abordagem visa garantir que as decisões tomadas maximizem as taxas de conversões, fundamentais para o sucesso contínuo da plataforma.

## Importação do dataset e das bibliotecas

In [19]:
# Importação do dataset
# https://drive.google.com/file/d/1K_2ZDsrhheRDKoHN2ynfLCPGPFiS2CZr/view?usp=drive_link
!gdown 1K_2ZDsrhheRDKoHN2ynfLCPGPFiS2CZr

Downloading...
From: https://drive.google.com/uc?id=1K_2ZDsrhheRDKoHN2ynfLCPGPFiS2CZr
To: /content/ITL-SI10-2024-P1-dataset.csv
100% 7.03k/7.03k [00:00<00:00, 22.8MB/s]


In [20]:
# Importação das bibliotecas
import pandas as pd
!pip install plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Transforma o dataset em DataFrame Pandas
df = pd.read_csv('ITL-SI10-2024-P1-dataset.csv')
df.head()

,Data,Visitantes Únicos,Sessões,Taxa de Rejeição (%),Páginas por Sessão,Principais Origens do Tráfego,Dispositivos,Eventos,Taxa de Conversão (%),Tempo Médio de Sessão (minutos)
0,2023-01-01,113.0,126.0,50.496714,1.834201,Direto,Desktop,8,1.952083,5.395516
1,2023-01-02,90.0,157.0,49.438007,1.955760,Social,Desktop,5,2.066180,4.680899
2,2023-01-03,92.0,185.0,49.800231,2.285052,Direto,Desktop,9,2.058858,5.972584
3,2023-01-04,130.0,206.0,50.251843,2.325464,Pago,Mobile,6,2.134617,4.705854
4,2023-01-05,123.0,155.0,48.070931,2.267006,Direto,Desktop,6,2.420236,5.835801


## Questões

### Questão 1

**Análise Exploratória de Dados**: Faça a exploração dos dados, utilizando gráficos interativos. Identifique correlações entre as variáveis e apresente essas relações em uma matriz de correlação focada nas variáveis mais relevantes para a conversão de vendas.

In [29]:
df2 = df.drop(['Data'], axis = 1)
df_encoded = pd.get_dummies(df2, columns=['Principais Origens do Tráfego', 'Dispositivos'])
df_encoded = df_encoded.replace({True: 1, False: 0})
df_encoded.head()

,Visitantes Únicos,Sessões,Taxa de Rejeição (%),Páginas por Sessão,Eventos,Taxa de Conversão (%),Tempo Médio de Sessão (minutos),Principais Origens do Tráfego_Direto,Principais Origens do Tráfego_Orgânico,Principais Origens do Tráfego_Pago,Principais Origens do Tráfego_Referência,Principais Origens do Tráfego_Social,Dispositivos_Desktop,Dispositivos_Mobile,Dispositivos_Tablet
0,113.0,126.0,50.496714,1.834201,8,1.952083,5.395516,1,0,0,0,0,1,0,0
1,90.0,157.0,49.438007,1.955760,5,2.066180,4.680899,0,0,0,0,1,1,0,0
2,92.0,185.0,49.800231,2.285052,9,2.058858,5.972584,1,0,0,0,0,1,0,0
3,130.0,206.0,50.251843,2.325464,6,2.134617,4.705854,0,0,1,0,0,0,1,0
4,123.0,155.0,48.070931,2.267006,6,2.420236,5.835801,1,0,0,0,0,1,0,0


In [30]:
import numpy as np

df_corr = df_encoded.corr()

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = df_corr.columns,
        y = df_corr.index,
        z = np.array(df_corr),
        text=df_corr.values,
        texttemplate='%{text:.2f}'
    )
)
fig.show()

A partir da matriz de correlação, fica visível que para o objetivo de aumentar as taxas de conversão, focando nessa variável na análise dos resultados, fica claro que:
- Correlação positiva: Visitantes únicos, Sessões, Páginas por Sessão e Tempo Médio de Sessão (minutos) representam uma conversão positiva, ou seja, quanto maior o valor de cada uma dessas variáveis, maior a taxa de conversão
- Correlação negativa: Taxa de Rejeição (%) demonstra que, quanto menor for essa taxa, maior será a taxa de conversão.

In [31]:
df['Data'] = pd.to_datetime(df['Data'])

daily_df = df.groupby(df['Data'].dt.date).agg({
    'Visitantes Únicos': 'sum',
    'Sessões': 'sum',
    'Páginas por Sessão': 'sum',
    'Tempo Médio de Sessão (minutos)': 'sum',
    'Taxa de Rejeição (%)' : 'sum'
}).reset_index()

daily_df.rename(columns={'Data': 'Date'}, inplace=True)

fig = px.line(daily_df, x='Date', y='Visitantes Únicos', title='Visitantes Únicos Over Time')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Visitantes Únicos')
fig.show()

fig = px.line(daily_df, x='Date', y='Sessões', title='Sessões Over Time')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Sessões')
fig.show()

fig = px.line(daily_df, x='Date', y='Páginas por Sessão', title='Páginas por Sessão Over Time')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Páginas por Sessão')
fig.show()

fig = px.line(daily_df, x='Date', y='Tempo Médio de Sessão (minutos)', title='Tempo Médio de Sessão (minutos) Over Time')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Tempo Médio de Sessão (minutos)')
fig.show()

fig = px.line(daily_df, x='Date', y='Taxa de Rejeição (%)', title='Taxa de Rejeição (%) Over Time')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Taxa de Rejeição (%)')
fig.show()

### Questão 2

**Simulação de Monte Carlo**: Implemente uma simulação de Monte Carlo para avaliar os possíveis efeitos de um novo redesign na plataforma. Analise como diferentes variáveis podem impactar os resultados e discuta os cenários que esta simulação pode prever, ajudando a empresa a tomar decisões mais informadas sobre o redesign.

In [34]:
data_inicio_mudancas = pd.Timestamp('2023-01-30')

df_antes = df[df['Data'] < data_inicio_mudancas]
df_depois = df[df['Data'] >= data_inicio_mudancas]

media_antes_rejeicao = df_antes['Taxa de Rejeição (%)'].mean()
media_depois_rejeicao = df_depois['Taxa de Rejeição (%)'].mean()
std_antes_rejeicao = df_antes['Taxa de Rejeição (%)'].std()
std_depois_rejeicao = df_depois['Taxa de Rejeição (%)'].std()

media_antes_trafego = df_antes['Visitantes Únicos'].mean()
media_depois_trafego = df_depois['Visitantes Únicos'].mean()
std_antes_trafego = df_antes['Visitantes Únicos'].std()
std_depois_trafego = df_depois['Visitantes Únicos'].std()

media_antes_sessoes = df_antes['Sessões'].mean()
media_depois_sessoes = df_depois['Sessões'].mean()
std_antes_sessoes = df_antes['Sessões'].std()
std_depois_sessoes = df_depois['Sessões'].std()

media_antes_pagina = df_antes['Páginas por Sessão'].mean()
media_depois_pagina = df_depois['Páginas por Sessão'].mean()
std_antes_pagina = df_antes['Páginas por Sessão'].std()
std_depois_pagina = df_depois['Páginas por Sessão'].std()

media_antes_tempo = df_antes['Tempo Médio de Sessão (minutos)'].mean()
media_depois_tempo = df_depois['Tempo Médio de Sessão (minutos)'].mean()
std_antes_tempo = df_antes['Tempo Médio de Sessão (minutos)'].std()
std_depois_tempo = df_depois['Tempo Médio de Sessão (minutos)'].std()

def simular_impacto(n_simulacoes):
    resultados_rejeicao = []
    resultados_trafego = []
    resultados_sessoes = []
    resultados_pagina = []
    resultados_tempo = []


    for _ in range(n_simulacoes):
        contribuicao_ui = np.random.uniform(0, 1)
        contribuicao_velocidade = 1 - contribuicao_ui

        ajuste_rejeicao = (media_depois_rejeicao - media_antes_rejeicao) * contribuicao_ui
        ajuste_trafego = (media_depois_trafego - media_antes_trafego) * contribuicao_ui
        ajuste_sessoes = (media_depois_sessoes - media_antes_sessoes) * contribuicao_ui
        ajuste_pagina = (media_depois_pagina - media_antes_pagina) * contribuicao_ui
        ajuste_tempo = (media_depois_tempo - media_antes_tempo) * contribuicao_ui

        nova_taxa_rejeicao = np.random.normal(media_antes_rejeicao + ajuste_rejeicao, std_depois_rejeicao)
        novo_volume_trafego = np.random.normal(media_antes_trafego + ajuste_trafego, std_depois_trafego)
        novas_sessoes = np.random.normal(media_antes_sessoes + ajuste_sessoes, std_depois_sessoes)
        novas_paginas = np.random.normal(media_antes_pagina + ajuste_pagina, std_depois_pagina)
        novo_tempo = np.random.normal(media_antes_tempo + ajuste_tempo, std_depois_tempo)

        resultados_rejeicao.append(nova_taxa_rejeicao)
        resultados_trafego.append(novo_volume_trafego)
        resultados_sessoes.append(novas_sessoes)
        resultados_pagina.append(novas_paginas)
        resultados_tempo.append(novo_tempo)

    return resultados_rejeicao, resultados_trafego, resultados_sessoes, resultados_pagina, resultados_tempo

n_simulacoes = 1000000
resultados_rejeicao_simulada, resultados_trafego_simulado, resultados_sessoes_simulado, resultados_pagina_simuladas, resultado_tempo_simulados = simular_impacto(n_simulacoes)

In [67]:
resultado = simular_impacto(n_simulacoes = 100000)
resultado

([40.14705978429944,
  39.24957656017076,
  46.989367052772145,
  34.95206826516299,
  35.72669498397009,
  30.197484357610193,
  22.057379893637425,
  46.88605620939433,
  37.72544785485046,
  37.8196034673369,
  40.366087804698935,
  25.802200941945916,
  28.678993283709115,
  31.998490919155447,
  38.4085102796327,
  35.952082197700015,
  39.69052809505799,
  23.324204747463945,
  38.352309041508505,
  22.13046085898009,
  32.513132306132476,
  44.9102973853213,
  19.719457075777946,
  39.28343272077293,
  48.8114486212612,
  46.58968857303022,
  28.343864026788797,
  37.36035738988619,
  37.16619865426194,
  47.579153562594136,
  38.82773218737427,
  35.11241096032769,
  32.626162010833404,
  45.30357785556558,
  40.64727899192721,
  36.70990300320642,
  26.442770798498245,
  36.139561553290946,
  25.970822051624218,
  30.739395494268923,
  37.73167790249846,
  36.27740825554623,
  40.39666776405413,
  28.150012890618182,
  32.92061387461395,
  30.541472727507134,
  40.349324147494

### Questão 3

**Análise de Sensibilidade**: Realize uma análise de sensibilidade para examinar como alterações em diferentes variáveis de entrada afetam as previsões da taxa de conversão.

In [53]:
X = df_encoded[["Taxa de Rejeição (%)", "Tempo Médio de Sessão (minutos)", "Sessões", "Páginas por Sessão", "Visitantes Únicos"]]
y = df_encoded["Taxa de Conversão (%)"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {np.sqrt(mse):.2f}")
print(f"MAE: {mae:.2f}")

RMSE: 0.17
MAE: 0.12


In [55]:
import pandas as pd
import numpy as np

sensibilidade_df = pd.DataFrame({
    'Taxa de Rejeição (%)': np.linspace(40, 60, 5),
    'Tempo Médio de Sessão (minutos)': np.linspace(1, 10, 5),
    'Sessões': np.linspace(100, 200, 5),
    'Páginas por Sessão': np.linspace(1,2, 5),
    'Visitantes Únicos': np.linspace(90, 100, 5)
})

predicted_conversions = model.predict(sensibilidade_df)

sensibilidade_df['Taxa de Conversão Prevista (%)'] = predicted_conversions

sensibilidade_df.head(10)

,Taxa de Rejeição (%),Tempo Médio de Sessão (minutos),Sessões,Páginas por Sessão,Visitantes Únicos,Taxa de Conversão Prevista (%)
0,40.0,1.00,100.0,1.00,90.0,2.630813
1,45.0,3.25,125.0,1.25,92.5,2.463876
2,50.0,5.50,150.0,1.50,95.0,2.191795
3,55.0,7.75,175.0,1.75,97.5,2.556333
4,60.0,10.00,200.0,2.00,100.0,2.749637


In [59]:
n_simulations = 1000
predicted_conversions = []

for _ in range(n_simulations):
    bootstrap_sample = resample(X, n_samples=len(X), replace=True, random_state=None)

    predicted_conversion = model.predict(bootstrap_sample)

    predicted_conversions.extend(predicted_conversion)

predicted_conversions_df = pd.DataFrame(predicted_conversions, columns=['Taxa de Conversão Prevista (%)'])

statistics_df = predicted_conversions_df.describe()
statistics_df

,Taxa de Conversão Prevista (%)
count,60000.000000
mean,4.502466
std,1.439488
min,2.155284
25%,3.246458
50%,4.502098
75%,5.728592
max,6.971389


In [60]:
fig = px.histogram(predicted_conversions_df, x='Taxa de Conversão Prevista (%)',
                   title='Distribuição da Taxa de Conversão Prevista',
                   labels={'Taxa de Conversão Prevista (%)': 'Taxa de Conversão (%)'},
                   nbins=30,
                   opacity=0.75,
                   color_discrete_sequence=['indianred'])

fig.update_layout(
    xaxis_title='Taxa de Conversão (%)',
    yaxis_title='Contagem',
    bargap=0.2,
    width=800,
    height=500
)
fig.show()

## Entrega

Siga esses passos para entregar sua prova.

1. Crie um repositório no GitHub para essa entrega.

2. Submeta seu desenvolvimento neste repositório.

3. Garanta que o repositório é publicamente acessível (eliminatório).

4. No Google forms da prova, submeta a URL do repositório.